# Libraries, API key, Data

In [8]:
!pip install -q langchain-community langchain-openai unstructured faiss-cpu langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.7 MB/s eta 0:00:00


In [4]:
%cd /content/drive/MyDrive/GenAI/RAG/Agentic RAG

/content/drive/MyDrive/GenAI/RAG/Agentic RAG


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
from google.colab import userdata
api_key = userdata.get('genai_course')

In [9]:
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.schema.output_parser import StrOutputParser

from typing import List
from typing_extensions import TypedDict
from IPython.display import display, Image

from langgraph.graph import StateGraph, END

In [10]:
# Specify the path to the Excel file containing the menu
file = 'dim sum montijo.xlsx'

# Create an instance of the loader with the specified mode
loader = UnstructuredExcelLoader(file, mode="elements")

# Load the data from the Excel file
data = loader.load()

In [11]:
# Initialize the embeddings model with the OpenAI API key
embeddings = OpenAIEmbeddings(openai_api_key=api_key)

# Create a vector store (FAISS) from the documents using the embeddings
db = FAISS.from_documents(data, embeddings)

# Process

In [ ]:
# TypedDict class to store information
class AgentState(TypedDict):
    start: bool
    conversation: int
    question: str
    answer: str
    topic: bool
    documents: list
    recursion_limit: int
    memory: list

In [ ]:
def greetings(state):
    # Greet the customer
    print("Hello! Welcome to the restaurant. I will be your waiter. How can I help you?")

    # Capture user input
    user_input = input()

    # Update the state with the customer's question and initialize conversation variables
    state['question'] = user_input
    state['conversation'] = 1
    state['memory'] = [user_input]

    return state

In [ ]:
def check_question(state):
    # Get the customer's question from the state
    question = state['question']

    # Define the system prompt to evaluate the question's appropriateness
    system_prompt = """
    You are a grader evaluating the appropriateness of a customer's question to a waiter or waitress in a restaurant.
    Assess if the question is suitable to ask the restaurant staff and if the customer shows interest in continuing the conversation.
    Respond with "True" if the question is appropriate for the staff or indicates the customer is asking a question or giving you information.
    Otherwise respond with "False".
    Provide only "True" or "False" in your response.
    """

    # Create a prompt template for formatting
    TEMPLATE = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "User question: {question}"),
    ])

    # Format the prompt with the customer's question
    prompt = TEMPLATE.format(question=question)

    # Initialize the language model
    model = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

    # Invoke the model with the prompt
    response_text = model.invoke(prompt)

    # Update the state with the model's decision
    state['topic'] = response_text.content.strip()

    return state

In [ ]:
# Function to check if the LLM model decides if the question is on topic on top
def topic_router(state):
  topic = state['topic']
  if topic == "True":
    return "on_topic"
  else:
    return "off_topic"

In [ ]:
def off_topic_response(state):
    # Check if it's the first conversation turn
    if state['conversation'] <= 1:
        state['answer'] = "\nI apologize, I can't answer that question. I can only answer questions about the menu in this restaurant."
        print(state['answer'])
    else:
        state['answer'] = "\nHappy to help."
        print(state['answer'])

In [ ]:
def retrieve_docs(state):
    # Combine the conversation history into a single string
    memory = " ".join(state['memory'])

    # Retrieve the top 5 relevant documents based on similarity to the conversation history
    docs_faiss = db.similarity_search(memory, k=5)

    # Store the retrieved documents' content in the state
    state['documents'] = [doc.page_content for doc in docs_faiss]

    return state

In [ ]:
def generate(state):
    # Initialize the language model
    model = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

    # Extract necessary information from the state
    question = state['question']
    documents = state['documents']
    memory = state['memory']

    # Define the system prompt for the waiter
    system_prompt = """
    You are a waiter at a restaurant tasked with answering customer's questions about the menu.
    Answer the question in the manner of a waiter, avoiding being too verbose or too brief.
    Do not include "waiter" or refer to yourself explicitly in your answer.
    """

    # Create a prompt template for formatting
    TEMPLATE = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "Context: {documents}\nConversation history so far: {memory}\nCustomer question: {question}"),
    ])

    # Format the prompt with the retrieved documents, conversation history, and customer's question
    prompt = TEMPLATE.format(documents=documents, memory=memory, question=question)

    # Invoke the model to generate an answer
    response_text = model.invoke(prompt)

    # Store the generated answer in the state
    state["answer"] = response_text.content.strip()

    return state

In [ ]:
def improve_answer(state):
    # Extract necessary information from the state
    question = state['question']
    answer = state['answer']
    memory = state['memory']

    # Define the system prompt for refining the answer
    system = """
    As a waiter, review and refine the response to a customer's question. Your task is to:

    1. Ensure the answer is appropriate, friendly, and informative.
    2. Edit or remove parts of the answer as needed, without adding new information.
    3. Maintain a polite, professional, and attentive tone.
    4. Provide only the improved answer, without any introductory phrases or commentary.
    5. Conclude the response with an open-ended question to invite further inquiries or address additional needs.
    6. Consider the conversation history to be more informative and useful.
    7. Include line breaks (`\n`) at the end of each sentence or logical break.

    Deliver a refined response that enhances the customer's experience and reflects the restaurant's commitment to customer service.
    """

    # Create a prompt template for formatting
    TEMPLATE = ChatPromptTemplate.from_messages([
        ("system", system),
        ("human", "Customer question: {question}\nConversation history: {memory}\nWaiter's initial answer: {answer}"),
    ])

    # Initialize the language model
    model = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

    # Format the prompt with the necessary information
    prompt = TEMPLATE.format(question=question, memory=memory, answer=answer)

    # Invoke the model to improve the answer
    response_text = model.invoke(prompt)

    # Update the state with the improved answer
    state['answer'] = response_text.content.strip()

    # Display the improved answer
    print('\n')
    print(state['answer'])

    # Append the improved answer to the conversation history
    state['memory'].append(state['answer'])

    return state

In [ ]:
def further_question(state):
    # Prompt the customer for further input
    print('\n')
    user_input = input()

    # Update the state with the new question and increment the conversation turn
    state['question'] = user_input
    state['conversation'] += 1
    state['memory'].append(user_input)

    return state

# Workflow

In [ ]:
# Initialize a StateGraph with the AgentState type
workflow = StateGraph(AgentState)


# Add the functions as nodes in the workflow
workflow.add_node("greetings", greetings)
workflow.add_node("check_question", check_question)
workflow.add_node("off_topic_response", off_topic_response)
workflow.add_node("retrieve_docs", retrieve_docs)
workflow.add_node("generate", generate)
workflow.add_node("improve_answer", improve_answer)
workflow.add_node("further_question", further_question)

# Set the entry point of the workflow to the greetings function
workflow.set_entry_point("greetings")

# Add conditional edges based on the topic_router function's output
workflow.add_conditional_edges(
    "check_question",
    topic_router,
    {
        "on_topic": "retrieve_docs",
        "off_topic": "off_topic_response"
    }
)

# Define the sequence of steps in the workflow
workflow.add_edge("greetings", "check_question")
workflow.add_edge("retrieve_docs", "generate")
workflow.add_edge("generate", "improve_answer")
workflow.add_edge("improve_answer", "further_question")
workflow.add_edge("further_question", "check_question")

# Connect the off_topic_response node to the end of the workflow
workflow.add_edge("off_topic_response", END)

# Compile the workflow into an application
app = workflow.compile()

In [ ]:
# Display the workflow graph for visualization
display(Image(app.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
# Invoke the application with the initial state
result = app.invoke({"start": True}, {"recursion_limit": 50})

Hello! Welcome to the restaurant. I will be your waiter. How can I help you?
best dish for a very hungry man


For a very hungry man, I recommend the Arroz em Lótus, featuring sticky rice with chicken, shrimp, and mushrooms.  

It's a hearty dish that should satisfy any big appetite.  

If you're looking for something to share, the Siao Long Pao or our Gyoza selection are also great options, as they are both flavorful and filling!  

Is there anything else I can assist you with or any other preferences you have?


how much dose it cost?


The Arroz em Lótus costs €5.30.  

The Siao Long Pao and Gyozas are priced at €4.50 each.  

If you'd like more information on other dishes or need assistance with your order, please feel free to ask!


what drink do you recommend with it?


For pairing with the Arroz em Lótus, I recommend a glass of Lello White or Red wine, as it complements the dish beautifully.  

If you prefer a non-alcoholic option, our Cold Tea selections, like the White Tea wit

KeyboardInterrupt: Interrupted by user